We are testing the functionality of apply_zx_simplification

In [205]:
import cirq
import pyzx as zx
from fractions import Fraction
import numpy as np
import pyzx.simplify as simplify
from pyzx import extract

In [206]:
MAX_QUBITS = 3

def circuit_from_cirq(circuit: cirq.Circuit) -> zx.Circuit:
# Verified and debugged. See code_verification\circuit_from_cirq.ipynb
    '''
    This function converts a Cirq circuit to a PyZX circuit.
    It supports the following gates:
    - H (Hadamard)
    - X, Z (with phase)
    - CNOT (controlled-NOT)
    - CZ (controlled-Z)
    '''
    zx_circ = zx.Circuit(MAX_QUBITS)
    for i, moment in enumerate(circuit):
        for op in moment:
            gate = op.gate
            q = [q.x for q in op.qubits]
            if isinstance(gate, cirq.HPowGate) and np.isclose(gate.exponent, 1):
                zx_circ.add_gate("H", q[0])
            elif isinstance(gate, cirq.XPowGate):
                phase = Fraction(gate.exponent).limit_denominator(1000)
                zx_circ.add_gate("XPhase", q[0], phase=phase)
            elif isinstance(gate, cirq.ZPowGate):
                phase = Fraction(gate.exponent).limit_denominator(1000)
                zx_circ.add_gate("ZPhase", q[0], phase=phase)
            elif isinstance(gate, cirq.YPowGate):
                pass
                # We avoid using YPhase
                # phase = Fraction(gate.exponent).limit_denominator(1000)
                # zx_circ.add_gate("YPhase", q[0], phase=phase)
                # displays YPhase gate via combination of X and Z phases
            elif isinstance(gate, cirq.CNotPowGate) and np.isclose(gate.exponent, 1):
                zx_circ.add_gate("CNOT", q[0], q[1])
            elif isinstance(gate, cirq.CZPowGate) and np.isclose(gate.exponent, 1):
                zx_circ.add_gate("CZ", q[0], q[1])
            else:
                print(f"    -> Unsupported or unknown gate: {type(gate)}")
    return zx_circ

In [207]:
def pyzx_to_cirq(pyzx_circuit):
# Verified and debugged. See code_verification\pyzx_to_cirq.ipynb
    '''
    Converts a PyZX circuit to a Cirq circuit.
    Supports the following gates:
    - XPhase (X with phase)
    - ZPhase (Z with phase)
    - Hadamard (H)
    - CNOT (controlled-NOT)
    - CZ (controlled-Z)
    '''
    pi = 3.141592653589793
    ops = []
    def get_qubits_used():
        max_index = 0
        for gate in pyzx_circuit.gates:
            targets = []
            if hasattr(gate, 'target'):
                t = gate.target
                if isinstance(t, (list, tuple)):
                    targets.extend(t)
                else:
                    targets.append(t)
            if hasattr(gate, 'control'):
                targets.append(gate.control)
            if targets:
                max_index = max(max_index, max(targets))
        return max_index + 1
    num_qubits = pyzx_circuit.n_qubits if hasattr(pyzx_circuit, 'n_qubits') else get_qubits_used()
    cirq_qubits = [cirq.LineQubit(i) for i in range(num_qubits)]
    for gate in pyzx_circuit.gates:
        name = gate.name.upper()
        if name == "ZPHASE":
            q = gate.target if isinstance(gate.target, int) else gate.target[0]
            ops.append(cirq.rz(gate.phase * pi).on(cirq_qubits[q]))
        elif name == "XPHASE":
            q = gate.target if isinstance(gate.target, int) else gate.target[0]
            ops.append(cirq.rx(gate.phase * pi).on(cirq_qubits[q]))
        elif name == "HAD":
            q = gate.target if isinstance(gate.target, int) else gate.target[0]
            ops.append(cirq.H(cirq_qubits[q]))
        elif name == "CNOT":
            ctrl = gate.control
            tgt = gate.target if isinstance(gate.target, int) else gate.target[0]
            ops.append(cirq.CNOT(cirq_qubits[ctrl], cirq_qubits[tgt]))
        elif name == "CZ":
            ctrl = gate.control
            tgt = gate.target if isinstance(gate.target, int) else gate.target[0]
            ops.append(cirq.CZ(cirq_qubits[ctrl], cirq_qubits[tgt]))
        else:
            pass  # or: print(f"Unrecognized gate {name}, skipping.")
    return cirq.Circuit(ops)

In [208]:
def apply_zx_simplification(circuit: cirq.Circuit) -> cirq.Circuit:
# Verified. See code_verification/apply_zx_simplification.ipynb
# The functionality or method for simplification is based on 
# https://arxiv.org/abs/2003.01664
    '''
    Applies simplification to the Cirq circuit using PyZX.
    Uses the pyzx methods
    - match_w_fusion_parallel:  wire fusion.
                                See https://pyzx.readthedocs.io/en/latest/api.html#pyzx.rules.match_w_fusion_parallel

    - match_ids, remove_ids:    to find identity pairs and remove them.
                                See https://pyzx.readthedocs.io/en/latest/api.html#pyzx.rules.remove_ids

    - extract_circuit:          to extract the simplified circuit.
                                See https://arxiv.org/abs/2003.01664
    '''
    zx_circ = circuit_from_cirq(circuit)
    g = zx_circ.to_graph()
    simplify.match_w_fusion_parallel(g)  # fuse some gates

    matches = simplify.match_ids(g)      # find identity pairs and remove
    simplify.remove_ids(g, matches)      

    simplified = extract.extract_circuit(g) # simplify

    cirq_circ = pyzx_to_cirq(simplified)
    if len(list(cirq_circ.all_operations())) == 0:
        return circuit  # fallback if too much was removed
    return cirq_circ

In [209]:
q0, q1, q2 = cirq.LineQubit.range(3)
circuit = cirq.Circuit(
    cirq.Z(q0),
    cirq.H(q1),
    cirq.H(q2),
    cirq.CZ(q2, q1),
    cirq.rx(3.14/2)(q0),
    cirq.rz(3.14)(q1),
    cirq.CNOT(q0, q1),
)

print("THE CIRCUIT:")
print(circuit)

print("THE SIMPLIFIED CIRCUIT from apply_zx_simplification():")
simplified_circuit = apply_zx_simplification(circuit)
print(simplified_circuit)

THE CIRCUIT:
0: ───Z───Rx(0.5π)────────────────@───
                                  │
1: ───H───@──────────Rz(0.999π)───X───
          │
2: ───H───@───────────────────────────
THE SIMPLIFIED CIRCUIT from apply_zx_simplification():
0: ───Rz(π)───H───Rz(0.499π)───H────────────────────@───
                                                    │
1: ───H───────H───@────────────H───Rz(0.999π)───H───@───
                  │
2: ───H───────H───@─────────────────────────────────────
